In [18]:
import numpy as np
import pandas as pd
import random as rand

import requests
import json
import re
from json import loads

In [29]:
bus_station = pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='euc-kr')
bus_station = bus_station.sample(n=100)
bus_station

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
11100,124900103,25584,길동초등학교,127.146380,37.538756
6686,115900078,16812,덕원여자고등학교.덕원예술고등학교,126.832207,37.549261
904,102900040,3592,용암경로당입구,126.986992,37.541091
3810,110000030,11130,두산힐스빌아파트,127.078930,37.621365
8898,120000673,21379,청림동현대아파트입구,126.957231,37.491016
...,...,...,...,...,...
5417,113000121,14212,서강대학교후문,126.944050,37.550476
7932,118000276,19367,크라운사거리,126.905338,37.508363
7638,117900161,18864,시흥동은행나무,126.909319,37.450794
8767,120000126,21228,보라매갑을아파트,126.922020,37.489879


In [62]:
len(pd.read_csv('서울시버스정류소좌표데이터(2022.08.24).csv', encoding='euc-kr'))

11167

In [64]:
# Starting point는 0 부여
loc0_name = '강남역'
loc0_X = '127.028361548'
loc0_Y = '37.496486063'

In [65]:
# 목적지는 1부터 네이밍;; 목적지 개수는 (num-1) 개

num = 30 # 총 노드 개수

for i in range(1,num): # 1~(n-1)
    globals()[f'loc{i}_name'] = bus_station.iloc[i,2]
    globals()[f'loc{i}_X'] = bus_station.iloc[i,3]
    globals()[f'loc{i}_Y'] = bus_station.iloc[i,4]

In [66]:
print(loc1_name, loc1_X, loc1_Y)

덕원여자고등학교.덕원예술고등학교 126.8322065733 37.5492613495


In [67]:
url = "https://apis.openapi.sk.com/tmap/matrix?version=1"

payload = {
    "origins": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "destinations": [
        {
            "lon": str(globals()[f'loc{i}_X']),
            "lat": str(globals()[f'loc{i}_Y']),
        } for i in range(num)
    ],
    "transportMode": "car",
    "metric": "Recommendation"
}
headers = {
    "Accept": "application/json",
    "appKey": "l7xxcdd1a30a6b34450a881f8083994a8cd4",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

In [68]:
jsonObj = json.loads(response.text)
jsonObj

{'meta': {'status': 'Ok',
  'message': 'Ok',
  'moduleVersion': '',
  'mapVersion': '20220920',
  'elapsedTime': 0,
  'realTimeTrafficDateTime': '',
  'patternTrafficDate': '',
  'customizationDateTime': '',
  'algorithm': '',
  'metric': 'Recommendation',
  'requestId': '8263daea-9a9a-4271-b240-14c84f309c64',
  'gasStationPriceDateTime': '',
  'transportMode': 'Car',
  'avoid': []},
 'origins': [{'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.496486063, 'longitude': 127.028361548},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.5492613495, 'longitude': 126.8322065733},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '',
   'coordinate': {'latitude': 37.5410908927, 'longitude': 126.9869921147},
   'rpFlag': ''},
  {'link': {'linkId': '0', 'direction': 'TwoWay'},
   'poiId': '',
   'name': '

In [69]:
jsonObj['matrixRoutes'][0]

{'status': 'Ok',
 'originIndex': 0,
 'destinationIndex': 0,
 'cost': 0,
 'duration': 0,
 'distance': 0.0,
 'toll': False}

In [70]:
duration_lst = []
distance_lst = []
for i in range(num**2):
    duration_lst.append(jsonObj['matrixRoutes'][i]['duration'])
    distance_lst.append(jsonObj['matrixRoutes'][i]['distance'])
duration = np.array(duration_lst).reshape(num,num)
distance = np.array(distance_lst).reshape(num,num)

In [71]:
print(duration.shape)
duration

(30, 30)


array([[   0, 3219, 1426, 3031, 2026, 3946, 3034, 3291, 2401, 2797, 3800,
        2478, 4825, 1738, 2341, 3365,  794, 2354, 3183, 2815,  704, 3034,
        3339, 1795, 2971, 2204, 2377, 2783, 3724, 1433],
       [3082,    0, 2500, 4522, 2300, 4289, 4524, 1486, 1877, 1369, 4651,
        4639, 4985, 4088, 3049, 4387, 3756, 3201, 2471, 4693, 4029,  864,
        4872, 3407, 1529, 2541, 1070, 2155, 2283, 2821],
       [1758, 2846,    0, 2797, 1459, 3712, 2800, 2007, 1673, 2141, 3565,
        3210, 3984, 2659, 1817, 3130, 2525, 2360, 2082, 3093, 2600, 2437,
        3105, 1270, 1687, 1402, 2005, 1198, 2373, 1727],
       [2905, 4219, 2758,    0, 3757, 1137,  359, 3122, 3579, 3568, 1281,
        2135, 1970, 2117, 2168,  677, 3000, 4164, 2503, 1197, 2233, 3850,
         618, 2157, 2995, 2535, 3346, 3333, 2754, 4047],
       [1638, 2238, 1244, 3261,    0, 4176, 3263, 1941, 1195, 1870, 4029,
        3488, 5054, 2516, 2660, 3594, 1588,  972, 2321, 3557, 2077, 2054,
        3569, 1952, 1621, 2002, 

In [77]:
print(distance.shape)
distance

(30, 30)


array([[    0. , 23744.7,  7916. , 19225.6,  8278.3, 24364.9, 19829.6,
        20798. , 12042.1, 16163.8, 22878.7, 20827.6, 27658. ,  9226.8,
        11143.4, 21284.8,  4659.5, 16841.5, 15228.1, 20662.9,  3910.1,
        20236.4, 21035.2,  9106.6, 19443.2, 10190. , 18873.4, 14566. ,
        22020.6,  5190.5],
       [25447.6,     0. , 20346.6, 29799.9, 18606.2, 41936.5, 30403.9,
        10202.3, 13027.1,  7427.4, 28129.6, 43847.5, 29224. , 33582.9,
        20362.6, 45996.3, 30398.9, 22034.2, 13848.3, 35468.2, 30290. ,
         4012.5, 31618.4, 25306.1, 10137.3, 17536.5,  5782.5, 15531.1,
        12714.7, 20385.6],
       [ 8770.9, 22947.4,     0. , 19650.6,  9783.8, 24789.9, 20254.6,
        16561. , 10385.2, 14305.6, 23303.7, 27846.1, 17798.1, 17581.5,
         7942.4, 21709.8, 12592.6, 13211.8,  9486.9, 19382.3, 14288.6,
        20608.1, 21460.2,  5905.6, 15206.2,  4170.4, 18076.1,  4432.4,
         9638.8,  8967.2],
       [23726.1, 29186.1, 15810.4,     0. , 25067.5,  6051.9,  1845

In [78]:
np.save('강남역_경로매트릭스(분)',duration)

In [79]:
np.save('강남역_경로매트릭(미터)',distance)